In [ ]:
import json
import re
import sys
import os
import pandas as pd
import ast
# from pandas import json_normalize
from tqdm import tqdm
from collections import Counter

In [ ]:
import sys
!{sys.executable} -m pip install pandas

# Select your output and input Path

In [31]:
output_path="/home/VolunteerismTransfer/data/new/data/"
base_path="/home/VolunteerismTransfer/data/"

# Read and Process All Labeled TREC DATA 2018- 2019-2020A

In [ ]:
def label_volunteer_tweets(postCategories):
    # if 'Volunteer' in postCategories and 'Donations' in postCategories and 'GoodsServices' in postCategories:
    #     return 'Vol-Don-Req'
    # elif 'Volunteer' in postCategories and 'Donations' in postCategories:
    #     return 'Vol-Don'
    # elif 'Volunteer' in postCategories and 'GoodsServices' in postCategories:
    #     return 'Vol-Req'
    # elif 'Donations' in postCategories and 'GoodsServices' in postCategories:
    #     return 'Don-Req'
    if 'Volunteer' in postCategories or 'Donations' in postCategories or 'GoodsServices' in postCategories:
        return 'Volunteer'
    # elif 'Donations' in postCategories:
    #     return 'Donations'
    # elif 'GoodsServices' in postCategories:
    #     return 'GoodServices'
    else:
        return "Non_Volunteer"

In [ ]:
def trunc_excessive_data_prop(data):
    for index, row in data.iterrows():
        if row['entities.hashtags'] != []:
            data.loc[index, "hashtags"] = str([element['text'] for element in json.loads(row['entities.hashtags'])])
        if row['entities.urls'] != []:
            if(row.eventid == 'hurricaneFlorence2018' ):
                data.loc[index, "urls"] = str(
                    [element['expandedURL'] for element in json.loads(row['entities.urls'])])
            else:
                data.loc[index, "urls"] = str([element['expanded_url'] for element in json.loads(row['entities.urls'])])
            if row['entities.user_mentions'] != []:
                data.loc[index, "user_mentions.id"] = str([element['id'] for element in json.loads(row['entities.user_mentions'])])
                data.loc[index, "user_mentions.name"] = str([element['name'] for element in json.loads(row['entities.user_mentions'])])



    data.drop(
            columns=['entities.hashtags', 'entities.urls','entities.user_mentions'],inplace=True)

    return data

In [ ]:
def merge_label_unlable():

    old_labels=pd.read_json(base_path+"TRECIS_2018_2019-labels.json", orient='records')
    old_labels.rename(columns={"postCategories": "categories","postPriority":"priority"}, inplace=True)
    old_labels = old_labels[['postID', 'categories', 'priority']]
    l=old_labels.groupby(['postID'])['categories'].agg('sum')
    old_labels=old_labels.drop(columns=['categories'])
    old_labels=pd.merge(l,all_trec, on='postID', how='inner')


    new_label = pd.read_json(base_path+"label_2020.json", orient='records', lines=True)
    new_label = new_label[['postID', 'categories', 'priority']]
    l=new_label.groupby(['postID'])['categories'].agg('sum')
    new_label=new_label.drop(columns=['categories'])
    new_label=pd.merge(l,all_trec, on='postID', how='inner')

    labels= old_labels.append(new_label)

    unlabel = pd.read_json(base_path+"out/all-2018-2019-2020.json", orient='records', lines=True)
    unlabel.rename(columns={"id": "postID"}, inplace=True)
    labels.drop_duplicates(['postID'], keep='first', inplace=True)
    labels['categories']=labels.groupby(['postID'])['categories'].agg('sum').values
    labels['categories']=labels.groupby(['postID'])['categories'].agg('sum').values
    unlabel.drop_duplicates(['postID'], keep='first', inplace=True)

    result = pd.merge(labels,unlabel, on='postID', how='inner')
    result['volunteerLabels'] = [label_volunteer_tweets(x) for x in result['categories']]
    # volunteer_df= result.query('volunteerLabels != "Non_Volunteer"')
    volunteer_df=trunc_excessive_data_prop(result)
    volunteer_df.to_json(output_path+"labeled_TREC-2018-2019-2020A.json", orient='records', lines=True)
    return  volunteer_df



In [ ]:
trec_data=merge_label_unlable()
trec_data['src']='trec'
trec_data.drop(columns=['user_mentions.id','user.id','user.name','urls','hashtags','priority','categories'],inplace=True)

In [ ]:
trec_data= pd.read_json(output_path+"labeled_TREC-2018-2019-2020A.json", orient='records', lines=True)
trec_data['src']='trec'

In [ ]:
trec_data.shape

# Read and Process  CrisisNLP  data

In [ ]:
def label_CrissNLP_volunteer_tweets_2(label):
    if 'Shelter and supplies' in label:
        return 'Volunteer'
    if 'Money' in label:
            return 'Volunteer'
    if 'Volunteer or professional services' in label:
            return 'Volunteer'
    if 'Humanitarian Aid Provided' in label:
            return 'Volunteer'
    if 'Needs of those affected' in label:
            return 'Volunteer'
    if 'Donations of money' in label:
            return 'Volunteer'
    if 'Donations of supplies and/or volunteer work' in label:
            return 'Volunteer'
    if 'Donation needs or offers or volunteering services' in label:
            return 'Volunteer'
    else:
        return "Non_Volunteer"

In [ ]:
def get_crisisNLP_Evnet(filename):
    filename=filename.lower()
    if filename.startswith("2014_California_Earthquake".lower()):
        return "2014_California_Earthquake"
    elif filename.startswith("2014_chile_earthquake".lower()) :
        return "2014_chile_earthquake"
    elif filename.startswith("2014_ebola"):
        return "2014_ebola"
    elif filename.startswith("2014_Hurricane_Odile_Mexico".lower()):
        return "2014_Hurricane_Odile_Mexico"
    elif filename.startswith("2014_Iceland_Volcano".lower()):
        return "2014_Iceland_Volcano"
    elif filename.startswith("2014_India_floods".lower()):
        return "2014_India_floods"
    elif filename.startswith("2014_MERS".lower()):
        return "2014_MERS"
    elif filename.startswith("2014_Middle_East_Respiratory_Syndrome".lower()):
        return "2014_Middle_East_Respiratory_Syndrome"
    elif filename.startswith("2014_Pakistan_floods".lower()):
        return "2014_Pakistan_floods"
    elif filename.startswith("2014_Philippines_Typhoon_Hagupi".lower()):
        return "2014_Philippines_Typhoon_Hagupi"
    elif filename.startswith("2014_Philippines_Typhoon_Hagupi".lower()) or filename.startswith("2014_Typhoon_Hagupit_en".lower()):
        return "2014_Philippines_Typhoon_Hagupi"
    elif filename.startswith("2015_Cyclone_Pam".lower()):
        return "2015_Cyclone_Pam"
    elif filename.startswith("2015_Nepal_Earthquake".lower()):
        return "2015_Nepal_Earthquake"
    elif filename.startswith("Landslides_Worldwide".lower()):
        return "Landslides_Worldwide"
    else:
        return "NA"

In [ ]:
def get_all_CrisisNLP_Data(file_path):
    data = pd.DataFrame()


    for filename in os.listdir(file_path):
        try:
            if filename.endswith('tsv'):
                df = pd.read_csv(file_path + filename,sep='\t')
            else:
                df = pd.read_csv(file_path + filename)
            df.rename(columns=lambda x: x.strip(), inplace=True)
            eventid=get_crisisNLP_Evnet(filename)
            df['volunteerLabels'] = [label_CrissNLP_volunteer_tweets_2(x) for x in df['label']]
            df.rename(columns={"tweet_id": "postID", "tweet_text": "text"}, inplace=True)
            df = df[['postID','text', 'volunteerLabels']]

            for index, row in df.iterrows():
                temp=row
                temp['user_mentions.name'] =str(regex.findall(row.text))
                temp['eventid'] = eventid
                data = data.append(temp)

        except:
            print("Unexpected error:", sys.exc_info()[0])
            print(filename)
            


In [ ]:
crissi_nlp_data=get_all_CrisisNLP_Data(base_path+"CrisisNLP_data/")
crissi_nlp_data['src']='crisis_nlp'

In [ ]:
crisisNLP_data=pd.read_json(output_path+"all_crisisNLP.json",orient='records', lines=True)
crisisNLP_data['src']='crisis_nlp'

In [ ]:
crisisNLP_data.shape

# Merge TREC IS  and CrisisNLP Data

In [ ]:
def label_eventtype(eventid):
    if 'earthquake' in eventid.lower():
        return 'earthquake'
    elif 'hurricane' in eventid.lower() or 'typhoon' in eventid.lower() or 'cyclone' in eventid.lower() or 'tornado' in eventid.lower():
        return 'hurricane/typhoon/cyclone/tornado'
    elif 'flood' in eventid.lower():
        return 'flood'
    elif 'wildfire' in eventid.lower() or 'bushfire' in eventid.lower() or 'fire' in eventid.lower():
        return 'wildfire/bushfire'
    elif 'bombing' in eventid.lower():
        return 'bombing'
    elif 'shooting' in eventid.lower():
        return 'shooting'
    elif 'explosion' in eventid.lower():
        return 'explosion'
    elif 'outbreak' in eventid.lower():
        return 'outbreak'
    elif 'collapse' in eventid.lower():
        return 'collapse'
    else:
        return 'general'

In [ ]:
labeled_data=crisisNLP_data.append(trec_data)
labeled_data['event_type'] = [label_eventtype(x) for x in labeled_data['eventid']]
labeled_data=labeled_data[["postID","eventid","event_type","text","volunteerLabels","src"]]
labeled_data.to_json(output_path+"Labeled.json",orient='records', lines=True)

In [ ]:
labeled_data['l'] = [convert_label(x) for x in labeled_data['volunteerLabels']]
labeled_data.query('l == 1').shape

# Get and Process NGO (Top Accounts) Tweets

In [ ]:
top_account_data = pd.read_json(output_path + 'top-account_data.json', orient='records', lines=True)
top_account_data['eventid'] = 'general'
top_account_data['volunteerLabels'] = 'Volunteer'
top_account_data['src'] = 'top-accounts'
top_account_data['event_type'] = 'general'
# data.drop(columns=['user_mentions.name'],inplace=True)
# data = data.append(top_account_data)


In [ ]:
labaled_with_NGO_Data=top_account_data.append(labeled_data)
labaled_with_NGO_Data.to_json(output_path+"NGO+Labeled Data.json",orient='records', lines=True)

# Get and Process Random TREC IS  Tweets

In [ ]:
def get_unlabled_Data(file_path):
    data = pd.DataFrame()


    for filename in os.listdir(file_path):

            df = pd.read_json(file_path + filename, orient='records', lines=True)
            df.rename(columns=lambda x: x.strip(), inplace=True)
            eventid=filename.replace(".json", "")
            df.rename(columns={"id": "postID", "full_text": "text"}, inplace=True)
            df = df[['postID','text']]
            df['eventid']=eventid
            data = data.append(df)


    return data



In [ ]:
random_data=get_unlabled_Data(base_path+"random_unlabeled_sample/")
random_data['volunteerLabels'] = 'unkonwn'
random_data['event_type'] = [label_eventtype(x) for x in random_data['eventid']]
random_data['src']='random'
random_data.to_json(output_path+"random_data.json", orient='records', lines=True)

In [ ]:
labaled_with_Random_Data=random_data.append(labeled_data)
labaled_with_Random_Data.to_json(output_path+"Random+Labeled Data.json",orient='records', lines=True)

# Merge RAndom , NGO and Labeled Data

In [ ]:
labaled_with_Random_NGO_Data=labaled_with_Random_Data.append(top_account_data)
labaled_with_Random_NGO_Data.to_json(output_path+"NGO+Random+Labeled Data.json",orient='records', lines=True)

In [ ]:
labeled_data.shape

In [ ]:
random_data.shape

In [ ]:
top_account_data.shape

In [ ]:
labaled_with_Random_NGO_Data.shape